The code of this notebook is the initial raw version of `make_dataset.py` which is located in `src/data`, so this notebook must not be executed.

In [6]:
import numpy as np
import pandas as pd
import pyhere

In [7]:
dir_data_raw = pyhere.here().resolve().joinpath("data", "raw")
dir_data_interim = pyhere.here().resolve().joinpath("data", "interim")
dir_data_external = pyhere.here().resolve().joinpath("data", "external")


In [8]:
csv_power_plants = pd.read_csv(dir_data_interim/"power_plants_with_generation_transformed.csv", index_col=[0])
max_index_csv_power_plants = len(csv_power_plants.index)
max_index_csv_power_plants
# df_sample = csv_power_plants.loc[0:30]
# df_sample.describe()
# max_index_sample_csv_power_plants = len(df_sample.index)
# max_index_sample_csv_power_plants

# df_sample

34936

In [4]:

# csv_power_plants
# with pd.read_csv(dir_data_interim/"power_plants_with_generation_transformed.csv", index_col=[0], iterator = True, chunksize= 50) as reader:
#     # print(reader.get_chunk(5))
#     for chunk in reader.:
#         print(chunk)
#     # print(reader.get_chunk(6))
# # reader

# sample_index_reference = 0

# while sample_index_reference + 5 <= (max_index_sample_csv_power_plants - 1):
#     try:
#         df_transformed_data_combined_with_nasa = pd.read_csv(dir_data_external/"transformed_data_combined_with_nasa.csv", index_col=['index'] )
#         sample_index_reference = sample_index_reference + 5
#     except FileNotFoundError: 
#         pass

#     top_to_iterate = None

#     if (sample_index_reference + 4) <= max_index_sample_csv_power_plants:
#         top_to_iterate = sample_index_reference + 4 
#     else:
#         top_to_iterate = max_index_sample_csv_power_plants -1
#     sample_lat_lon = df_sample.loc[sample_index_reference:top_to_iterate, ["latitude", "longitude"]]
#     print(sample_lat_lon)
#     print(f'{sample_index_reference}:{top_to_iterate} de {max_index_sample_csv_power_plants - 1}')
# sample_lat_lon = df_sample.loc[35:41, ["latitude", "longitude"]]
# sample_lat_lon

# LOOP SINGLE POINT

In [14]:
import os, json, requests
from io import StringIO
import certifi
import urllib3
from time import sleep

index_reference = 0
rows_chunk = 60
seconds_to_sleep = 0
url_parameters = ["ALLSKY_SFC_SW_DWN",
                    "CLRSKY_SFC_SW_DWN",
                    "ALLSKY_SFC_SW_DIFF",
                    "ALLSKY_SFC_SW_UP",
                    "ALLSKY_SFC_LW_DWN",
                    "ALLSKY_SFC_LW_UP",
                    "ALLSKY_SFC_SW_DNI",
                    # "ALLSKY_SFC_SW_DNI_MAX_RD",
                    "ALLSKY_SFC_SW_UP_MAX",
                    "CLRSKY_SFC_SW_DIFF",
                    "CLRSKY_SFC_SW_DNI",
                    "CLRSKY_SFC_SW_UP",
                    #"ALLSKY_KT",
                    "CLOUD_AMT_DAY",
                    "WS10M_MAX_AVG",
                    "WS50M_MAX_AVG",
                    "WS50M",
                    "WS50M_RANGE_AVG",
                    "WS10M",
                    "WS10M_RANGE_AVG"
                ]
ORDINAL_COLUMNS =   [
                '1st(min)',
                '2nd',
                '3rd',
                '4th',
                '5th',
                '6th',
                '7th',
                '8th',
                '9th',
                '10th',
                '11th',
                '12th(max)'
            ]
while index_reference + rows_chunk <= (max_index_csv_power_plants - 1):
# while index_reference + rows_chunk <= (2 - 1):
    try:
        df_transformed_data_combined_with_nasa = pd.read_csv(dir_data_external/"v6_transformed_data_combined_with_nasa.csv", index_col=['index'] )
        index_reference = df_transformed_data_combined_with_nasa.index.max() + 1
    except FileNotFoundError: 
        pass

    top_to_iterate = None

    if (index_reference + (rows_chunk - 1)) <= max_index_csv_power_plants:
        top_to_iterate = index_reference + (rows_chunk - 1)
    else:
        top_to_iterate = max_index_csv_power_plants -1
    sample_lat_lon = csv_power_plants.loc[index_reference:top_to_iterate, ["latitude", "longitude"]]
    print(f'{index_reference}:{top_to_iterate} de {max_index_csv_power_plants - 1}')



    locations = list(sample_lat_lon.to_records(index=False))

    NORTH_HEMISPHERE_MONTHS_SEASONS = dict()
    SOUTH_HEMISPHERE_MONTHS_SEASONS = dict()
    MONTHS_OF_YEAR = np.array(["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"])
    NORTH_HEMISPHERE_MONTHS_SEASONS["autumn"] = np.array(["OCT", "NOV", "DEC"])
    NORTH_HEMISPHERE_MONTHS_SEASONS["winter"] = np.array(["JAN", "FEB", "MAR"])
    NORTH_HEMISPHERE_MONTHS_SEASONS["spring"] = np.array(["APR", "MAY", "JUN"])
    NORTH_HEMISPHERE_MONTHS_SEASONS["summer"] = np.array(["JUL", "AUG", "SEP"])
    SOUTH_HEMISPHERE_MONTHS_SEASONS["spring"] = np.array(["OCT", "NOV", "DEC"])
    SOUTH_HEMISPHERE_MONTHS_SEASONS["summer"] = np.array(["JAN", "FEB", "MAR"])
    SOUTH_HEMISPHERE_MONTHS_SEASONS["autumn"] = np.array(["APR", "MAY", "JUN"])
    SOUTH_HEMISPHERE_MONTHS_SEASONS["winter"] = np.array(["JUL", "AUG", "SEP"])

    output = r""
    base_url = r"https://power.larc.nasa.gov/api/temporal/monthly/point?parameters={url_parameters}&community=RE&longitude={longitude}&latitude={latitude}&start=2012&end=2019&format=CSV&header=false"
    df_response = pd.DataFrame()

    http = urllib3.PoolManager(
        cert_reqs='CERT_REQUIRED',
        ca_certs=certifi.where()
    )

    aux_counter_index = index_reference

    filename_template = "v6_transformed_data_combined_with_nasa.csv"
    filename = filename_template
    for latitude, longitude in locations:
        api_request_url = base_url.format(longitude=longitude, latitude=latitude, url_parameters=','.join(url_parameters))
        response = http.request('GET', api_request_url, timeout=30.00).data.decode('utf-8')
        df_response_aux = pd.read_csv(StringIO(response))
        df_response_aux["latitude"] = latitude
        df_response_aux["longitude"] = longitude
        # print(df_response_aux)
    #     if longitude > 0:
    #         hemisphere_months_seasons = NORTH_HEMISPHERE_MONTHS_SEASONS
    #     else:
    #         hemisphere_months_seasons = SOUTH_HEMISPHERE_MONTHS_SEASONS
    #     for index, element in hemisphere_months_seasons.items():
    #         df_response_aux[index]= df_response_aux[element].mean(axis=1)

    #     df_response_aux.drop(columns= MONTHS_OF_YEAR, inplace = True)
        series_sorted_values_by_column = df_response_aux[MONTHS_OF_YEAR].apply(lambda row: row.sort_values().values, axis=1)
        df_aux_statistics = pd.DataFrame(series_sorted_values_by_column.values.tolist(), columns=ORDINAL_COLUMNS)
        df_aux_statistics['mean'] = np.around(df_aux_statistics[ORDINAL_COLUMNS].mean(axis=1),3)
        df_aux_statistics['std'] = np.around(df_aux_statistics[ORDINAL_COLUMNS].std(axis=1),3)
        df_aux_statistics['median'] = (df_aux_statistics['6th'] + df_aux_statistics['7th']) / 2
        df_aux_statistics['min'] = df_aux_statistics['1st(min)']
        df_aux_statistics['max'] = df_aux_statistics['12th(max)']
        df_aux_statistics.drop(columns= ORDINAL_COLUMNS, inplace = True)

        df_response_aux = pd.concat([df_response_aux, df_aux_statistics], axis=1)
        df_response_aux.drop(columns= MONTHS_OF_YEAR, inplace = True)
        df_response_aux = df_response_aux.pivot_table(index=["latitude", "longitude"], columns=["PARAMETER", "YEAR"])
        df_response_aux.columns = ["_".join(map(str, cols)) for cols in df_response_aux.columns.to_flat_index()]

        pd.concat([df_response_aux, csv_power_plants.loc[aux_counter_index, ['capacity_mw', 'primary_fuel_transformed']]], axis=1)

        print('x')
        if(df_response.empty):

            df_response = df_response_aux.copy()
        else:
            df_response = pd.concat([df_response,df_response_aux])

        aux_counter_index += 1


    df_response.reset_index(inplace = True)
    df_response.index += index_reference
    # TRANSFORMING AND COMBINING DATA
    try:

        df_response = pd.concat([df_response,df_transformed_data_combined_with_nasa])
        df_response.sort_index(inplace = True)
        del df_transformed_data_combined_with_nasa
    
    except NameError:
        pass

    df_response.to_csv(dir_data_external/filename, index_label='index')
    del df_response
    del df_response_aux
    print("Sleeping...")

540:599 de 34935
x
x


KeyboardInterrupt: 

In [ ]:
MONTHS_OF_YEAR

In [19]:
df_response_aux.columns

Index(['PARAMETER', 'YEAR', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL',
       'AUG', 'SEP', 'OCT', '\n       NOV', 'DEC', 'ANN', 'latitude',
       'longitude'],
      dtype='object')

In [18]:
df_response_aux[MONTHS_OF_YEAR].apply(lambda row: row.sort_values().values, axis=1)

KeyError: "['NOV'] not in index"

In [17]:
df_response_aux[MONTHS_OF_YEAR]

KeyError: "['NOV'] not in index"

In [13]:
ordinal_columns =   [
                '1st(min)',
                '2nd',
                '3rd',
                '4th',
                '5th',
                '6th',
                '7th',
                '8th',
                '9th',
                '10th',
                '11th',
                '12th(max)'
            ]

# df_response[ordinal_columns] = df_response_aux[MONTHS_OF_YEAR].apply(lambda row: row.sort_values().values, axis=1)

In [18]:
df_response_prueba = df_response_aux.copy()

In [40]:
series_sorted_values_by_column = df_response_aux[MONTHS_OF_YEAR].apply(lambda row: row.sort_values().values, axis=1)
df_middle = pd.DataFrame(series_sorted_values_by_column.values.tolist(), columns=ordinal_columns)

# df_response_prueba = pd.concat([df_response_prueba, pd.DataFrame(series_sorted_values_by_column.values.tolist(), columns=ordinal_columns)], axis=1)

In [41]:
df_middle['mean'] = np.around(df_middle[ordinal_columns].mean(axis=1),3)
df_middle['std'] = np.around(df_middle[ordinal_columns].std(axis=1),3)

In [42]:
df_middle

,1st(min),2nd,3rd,4th,5th,6th,7th,8th,9th,10th,11th,12th(max),mean,std
0,2.54,2.62,2.77,2.81,2.84,3.08,3.10,3.20,3.61,4.09,4.24,4.30,3.267,0.637
1,2.40,2.48,2.53,2.79,2.84,2.87,3.03,3.20,3.50,3.56,3.73,4.05,3.082,0.531
2,2.36,2.42,2.50,2.80,3.03,3.14,3.28,3.30,3.55,3.60,3.69,4.38,3.171,0.597
3,2.38,2.62,2.62,2.86,2.97,2.98,3.23,3.41,3.55,3.78,4.06,4.12,3.215,0.576
4,2.65,2.66,2.74,2.75,3.12,3.18,3.28,3.31,3.40,3.78,3.89,4.06,3.235,0.487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0.92,1.11,1.13,1.17,1.50,1.63,1.92,1.97,1.99,2.07,2.17,2.20,1.648,0.466
140,0.93,1.12,1.24,1.49,1.54,1.61,1.85,1.89,1.99,2.00,2.05,2.08,1.649,0.392
141,0.97,1.02,1.20,1.35,1.54,1.59,1.74,2.02,2.04,2.04,2.11,2.16,1.648,0.436
142,0.98,1.04,1.30,1.30,1.64,1.65,1.90,1.91,2.05,2.21,2.27,2.28,1.711,0.468


In [43]:
df_middle['median'] = (df_middle['6th'] + df_middle['7th']) / 2
df_middle['min'] = df_middle['1st(min)']
df_middle['max'] = df_middle['12th(max)']

In [44]:
df_middle.drop(columns= ordinal_columns, inplace = True)

In [45]:
df_middle

,mean,std,median,min,max
0,3.267,0.637,3.090,2.54,4.30
1,3.082,0.531,2.950,2.40,4.05
2,3.171,0.597,3.210,2.36,4.38
3,3.215,0.576,3.105,2.38,4.12
4,3.235,0.487,3.230,2.65,4.06
...,...,...,...,...,...
139,1.648,0.466,1.775,0.92,2.20
140,1.649,0.392,1.730,0.93,2.08
141,1.648,0.436,1.665,0.97,2.16
142,1.711,0.468,1.775,0.98,2.28


In [53]:
df_response_prueba

,PARAMETER,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,...,3rd,4th,5th,6th,7th,8th,9th,10th,11th,12th(max)
0,WS10M,2012,2.77,3.08,3.61,3.20,4.24,4.09,4.30,3.10,...,2.77,2.81,2.84,3.08,3.10,3.20,3.61,4.09,4.24,4.30
1,WS10M,2013,2.53,2.84,3.03,3.56,3.73,4.05,3.50,3.20,...,2.53,2.79,2.84,2.87,3.03,3.20,3.50,3.56,3.73,4.05
2,WS10M,2014,2.36,2.50,3.14,3.28,3.55,4.38,3.60,3.69,...,2.50,2.80,3.03,3.14,3.28,3.30,3.55,3.60,3.69,4.38
3,WS10M,2015,2.62,2.86,2.97,3.41,4.06,4.12,3.78,3.55,...,2.62,2.86,2.97,2.98,3.23,3.41,3.55,3.78,4.06,4.12
4,WS10M,2016,2.66,2.75,3.31,3.40,3.89,3.78,4.06,3.28,...,2.74,2.75,3.12,3.18,3.28,3.31,3.40,3.78,3.89,4.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,ALLSKY_SFC_SW_UP_MAX,2015,1.11,1.17,1.63,1.97,2.20,2.17,2.07,1.99,...,1.13,1.17,1.50,1.63,1.92,1.97,1.99,2.07,2.17,2.20
140,ALLSKY_SFC_SW_UP_MAX,2016,1.12,1.49,1.61,1.89,2.00,2.05,2.08,1.99,...,1.24,1.49,1.54,1.61,1.85,1.89,1.99,2.00,2.05,2.08
141,ALLSKY_SFC_SW_UP_MAX,2017,0.97,1.35,1.54,2.04,2.04,2.16,2.11,2.02,...,1.20,1.35,1.54,1.59,1.74,2.02,2.04,2.04,2.11,2.16
142,ALLSKY_SFC_SW_UP_MAX,2018,1.04,1.30,1.64,1.91,2.21,2.27,2.28,2.05,...,1.30,1.30,1.64,1.65,1.90,1.91,2.05,2.21,2.27,2.28
